## Load the Data

In [107]:
import pandas as pd
import numpy as np


## Inspect the Data

In [108]:
df = pd.read_csv('Airbnb_Open_Data.csv')

C:\Users\alexs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host name                       102193 non-null  object 
 5   neighbourhood group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

In [110]:
df.head(2)

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN


In [111]:
df.columns

Index(['id', 'NAME', 'host id', 'host_identity_verified', 'host name',
       'neighbourhood group', 'neighbourhood', 'lat', 'long', 'country',
       'country code', 'instant_bookable', 'cancellation_policy', 'room type',
       'Construction year', 'price', 'service fee', 'minimum nights',
       'number of reviews', 'last review', 'reviews per month',
       'review rate number', 'calculated host listings count',
       'availability 365', 'house_rules', 'license'],
      dtype='object')

In [112]:
dfc = df.copy()

## Handle Duplicates

In [113]:
duplicate_rows = dfc[dfc.duplicated()]
len(duplicate_rows)

541

In [114]:
dfc.drop_duplicates(inplace=True)


In [115]:
duplicate_rows = dfc[dfc.duplicated()]
len(duplicate_rows)

0

## Correct Data Types

In [116]:
dfc['price'] = pd.to_numeric(dfc['price'].str.replace('$', ''), errors='coerce')


C:\Users\alexs\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [117]:
print(dfc['price'].head())


0    966.0
1    142.0
2    620.0
3    368.0
4    204.0
Name: price, dtype: float64


In [118]:
dfc['service fee'] = pd.to_numeric(dfc['service fee'].str.replace('$', ''), errors='coerce')


C:\Users\alexs\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [119]:
print(dfc['service fee'].head())


0    193.0
1     28.0
2    124.0
3     74.0
4     41.0
Name: service fee, dtype: float64


In [120]:
# Convert 'last review' to datetime
dfc['last review'] = pd.to_datetime(df['last review'], errors='coerce')

# Convert 'id' and 'host id' to string
dfc['id'] = dfc['id'].astype(str)
dfc['host id'] = dfc['host id'].astype(str)

In [121]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102058 entries, 0 to 102057
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   id                              102058 non-null  object        
 1   NAME                            101808 non-null  object        
 2   host id                         102058 non-null  object        
 3   host_identity_verified          101769 non-null  object        
 4   host name                       101654 non-null  object        
 5   neighbourhood group             102029 non-null  object        
 6   neighbourhood                   102042 non-null  object        
 7   lat                             102050 non-null  float64       
 8   long                            102050 non-null  float64       
 9   country                         101526 non-null  object        
 10  country code                    101927 non-null  object 

## Handle Missing Values

In [122]:
print(dfc.isnull().sum())


id                                     0
NAME                                 250
host id                                0
host_identity_verified               289
host name                            404
neighbourhood group                   29
neighbourhood                         16
lat                                    8
long                                   8
country                              532
country code                         131
instant_bookable                     105
cancellation_policy                   76
room type                              0
Construction year                    214
price                              18059
service fee                          273
minimum nights                       400
number of reviews                    183
last review                        15832
reviews per month                  15818
review rate number                   319
calculated host listings count       319
availability 365                     448
house_rules     

In [123]:
# The median is less affected by outliers compared to the mean

dfc['price'].fillna(dfc['price'].median(), inplace=True)
dfc['service fee'].fillna(dfc['service fee'].median(), inplace=True)

In [124]:
dfc['reviews per month'].fillna(0, inplace=True)


In [125]:
dfc['host name'].fillna('Unknown', inplace=True)
dfc['house_rules'].fillna('No rules specified', inplace=True)

## Outlier Detection and Handling

In [126]:
upper_limit = dfc['price'].quantile(0.99)
lower_limit = dfc['price'].quantile(0.01)

# dfc = dfc[(dfc['price'] >= lower_limit) & (dfc['price'] <= upper_limit)]

In [127]:
upper_limit

988.0

In [128]:
lower_limit

61.0

## Handle Inconsistent Data